# Simple Ensemble For Regression Model

In [6]:
import pandas as pd
from sklearn import preprocessing
from neupy import environment
from sklearn.model_selection import train_test_split
from neupy import environment
from operator import itemgetter
import numpy as np
from neupy import algorithms, layers, estimators
from sklearn import datasets, grid_search
#from sklearn.model_selection import GridSearchCV


def scorer(actual, predicted):
    return estimators.rmse(predicted, actual)

training_df = pd.read_csv(
    "data_with_fields_removed/train_70.0_updated.csv")
training_X = training_df.loc[:, training_df.columns != " shares"]
training_Y = training_df.loc[:, " shares"]

test_df = pd.read_csv(
    "data_with_fields_removed/test_30.0_updated.csv")
testing_X = test_df.loc[:, test_df.columns != " shares"]
testing_Y = test_df.loc[:, " shares"]

print("Training shapes")
print(training_X.shape)
print(training_Y.shape)

print("Testing shapes")
print(testing_X.shape)
print(testing_Y.shape)

x_data_scaler = preprocessing.MinMaxScaler()
y_data_scaler = preprocessing.MinMaxScaler()

columns_to_scale = [" n_tokens_title", " n_tokens_content", " num_hrefs", " num_self_hrefs", " num_imgs", " num_videos", " average_token_length", " num_keywords", " kw_avg_min"," kw_avg_max", " kw_avg_avg"," self_reference_avg_sharess"]

x_data_scaler.fit(training_X.loc[:, columns_to_scale])
y_data_scaler.fit(training_Y.values.reshape(-1, 1))

training_X.loc[:, columns_to_scale] = x_data_scaler.transform(training_X.loc[:, columns_to_scale])
training_Y = y_data_scaler.transform(training_Y.values.reshape(-1, 1))

testing_X.loc[:, columns_to_scale] = x_data_scaler.transform(testing_X.loc[:, columns_to_scale])
testing_Y = y_data_scaler.transform(testing_Y.values.reshape(-1, 1))


Training shapes
(27750, 27)
(27750,)
Testing shapes
(11894, 27)
(11894,)


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Load the Pre-Trained Models

In [7]:
from sklearn.neural_network import MLPRegressor
from scipy import stats
from sklearn.grid_search import RandomizedSearchCV
from neupy import estimators
import _pickle

with open('regression_models/multi_layer_perceptron.pkl', 'rb') as fid:
    mlp = _pickle.load(fid)

with open('regression_models/grnn.pkl', 'rb') as fid:
    grnn = _pickle.load(fid)

In [3]:
models = [mlp, grnn]
y_ensemble_predicted = np.zeros(shape=(testing_X.shape[0], 1))
denom = 0

for model in models:
    print(model)
    y_predicted = model.predict(testing_X)
    if model.__class__ == grnn.__class__: # take care of neupy diff format
        y_predicted = y_predicted.ravel().transpose()
    mae = estimators.mae(y_predicted, testing_Y.ravel())
    denom = denom + (1.0 / (1+mae))
    y_ensemble_predicted = np.add(y_ensemble_predicted, ((1.0 / (1+mae)) * y_predicted))

ensemble_mae = estimators.mae(y_ensemble_predicted, testing_Y.ravel())
ensemble_mae = ensemble_mae / denom
print("Ensemble MAE = " + str(ensemble_mae))
actual_mae = y_data_scaler.inverse_transform(ensemble_mae)
print("Ensemble MAE (no. of shares) = " + str(actual_mae.squeeze()))

MLPRegressor(activation='tanh', alpha=0.215, batch_size='auto', beta_1=0.5,
       beta_2=0.76, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(1000, 100, 100), learning_rate='constant',
       learning_rate_init=0.0218, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
GRNN(verbose=True, step=None, show_epoch=None, shuffle_data=None, epoch_end_signal=None, train_end_signal=None, std=0.2)
Ensemble MAE = 0.003985574610490937
Ensemble MAE (no. of shares) = 2752.636725508333
